In [605]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  

In [606]:
# loading data
#a = pd.read_excel("3.xlsx")

In [607]:
#preventing from frequent loading 
df = a

In [608]:
#drop rows with at least 30 missing values
#df.dropna(thresh=df.shape[1] - 10, inplace=True)

In [609]:

# Drop column: 'L1_BloodGroup_First'
df = df.drop(columns=['L1_BloodGroup_First'])
# Drop rows with missing data in column: 'Outcome_InhospitalMortality'
df = df.dropna(subset=['Outcome_InhospitalMortality'])


In [610]:
# splitting internal and external validation
df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]
df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])

In [611]:
X = df_internal_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y  = df_internal_validation['Outcome_InhospitalMortality']

In [612]:
X_ex = df_external_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_ex  = df_external_validation['Outcome_InhospitalMortality']

In [613]:
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.2 , random_state = 42)

In [614]:
num =  [
    "symtpm_to_referral",
    "VS_O2satwithoutsupp",
    "VS_PR",
    "VS_diastolic BP",
    "VS_Systolic BP",
    "VS_RR",
    "VS_T",
    "LAB_WBC_1",
    "LAB_LYMPHH_1",
    "LAB_NEUT_1",
    "LAB_PLT_1",
    "LAB_HB_1",
    "LAB_MCV_1",
    "LAB_CR_1",
    "LAB_NA_First",
    "LAB_K_First",
    "LAB_ALKP_First",
    "LAB_ESR_First",
    "LAB_CPK_First",
    "LAB_PTT_First",
    "LAB_PT_First",
    "LAB_INR_First",
    "Demographic_Age"
]

In [615]:
cat = X.drop(columns= num).columns.tolist()

In [616]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

X_train_cat = imputer.fit_transform(np.array(X_train[cat]))
X_test_cat = imputer.fit_transform(np.array(X_test[cat]))
X_ex_cat = imputer.fit_transform(np.array(X_ex[cat]))

X_train_cat = pd.DataFrame(X_train_cat, columns=cat)
X_test_cat = pd.DataFrame(X_test_cat, columns=cat)
X_ex_cat = pd.DataFrame(X_ex_cat, columns=cat)

In [617]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=50, random_state=0)

X_train_num = imputer.fit_transform(X_train[num])
X_test_num = imputer.fit_transform(X_test[num])
X_ex_num = imputer.fit_transform(X_ex[num])

X_train_num = pd.DataFrame(X_train_num, columns=num)
X_test_num = pd.DataFrame(X_test_num, columns=num)
X_ex_num = pd.DataFrame(X_ex_num, columns=num)


In [618]:
X_train = pd.concat([X_train_cat, X_train_num], axis=1)
X_test = pd.concat([X_test_cat, X_test_num], axis=1)
X_ex = pd.concat([X_ex_cat, X_ex_num], axis=1)


In [619]:
from operator import itemgetter
def find_skewness(train, num):
    """
    Calculate the skewness of the columns and segregate the positive
    and negative skewed data.
    """
    skew_dict = {}
    for col in num:
        skew_dict[col] = train[col].skew()
    skew_dict = dict(sorted(skew_dict.items(),key=itemgetter(1)))
    positive_skew_dict = {k:v for (k,v) in skew_dict.items() if v>0}
    negative_skew_dict = {k:v for (k,v) in skew_dict.items() if v<0}
    return skew_dict, positive_skew_dict, negative_skew_dict
def add_constant(data, highly_pos_skewed):
    """
    Look for zeros in the columns. If zeros are present then the log(0) would result in -infinity.
    So before transforming it we need to add it with some constant.
    """
    C = 1
    for col in highly_pos_skewed.keys():
        if(col != 'SalePrice'):
            if(len(data[data[col] == 0]) > 0):
                data[col] = data[col] + C
    return data
def log_transform(data, highly_pos_skewed):
    """
    Log transformation of highly positively skewed columns.
    """
    for col in highly_pos_skewed.keys():
        if(col != 'SalePrice'):
            data[col] = np.log10(data[col])
    return data
def sqrt_transform(data, moderately_pos_skewed):
    """
    Square root transformation of moderately skewed columns.
    """
    for col in moderately_pos_skewed.keys():
        if(col != 'SalePrice'):
            data[col] = np.sqrt(data[col])
    return data
def reflect_sqrt_transform(data, moderately_neg_skewed):
    """
    Reflection and log transformation of highly negatively skewed 
    columns.
    """
    for col in moderately_neg_skewed.keys():
        if(col != 'SalePrice'):
            K = max(data[col]) + 1
            data[col] = np.sqrt(K - data[col])
    return data
"""
If skewness is less than -1 or greater than 1, the distribution is highly skewed.
If skewness is between -1 and -0.5 or between 0.5 and 1, the distribution is moderately skewed.
If skewness is between -0.5 and 0.5, the distribution is approximately symmetric.
"""
skew_dict, positive_skew_dict, negative_skew_dict = find_skewness(X_train, num)
moderately_pos_skewed = {k:v for (k,v) in positive_skew_dict.items() if v>0.5 and v<=1}
highly_pos_skewed = {k:v for (k,v) in positive_skew_dict.items() if v>1}
moderately_neg_skewed = {k:v for (k,v) in negative_skew_dict.items() if v>-1 and v<=0.5}
highly_neg_skewed = {k:v for (k,v) in negative_skew_dict.items() if v<-1}
'''Transform train data.'''
X_train = add_constant(X_train, highly_pos_skewed)
X_train = log_transform(X_train, highly_pos_skewed)
X_train = sqrt_transform(X_train, moderately_pos_skewed)
X_train = reflect_sqrt_transform(X_train, moderately_neg_skewed )
'''Transform test data.'''
X_test = add_constant(X_test, highly_pos_skewed)
X_test = log_transform(X_test, highly_pos_skewed)
X_test = sqrt_transform(X_test, moderately_pos_skewed)
X_test = reflect_sqrt_transform(X_test, moderately_neg_skewed )

X_ex = add_constant(X_ex, highly_pos_skewed)
X_ex = log_transform(X_ex, highly_pos_skewed)
X_ex = sqrt_transform(X_ex, moderately_pos_skewed)
X_ex = reflect_sqrt_transform(X_ex, moderately_neg_skewed )

d:\python\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [620]:
X_train

,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,Symptom_Chiver,Symptom_Mylagia,Symptom_Weakness,Symptom_LOC,Symptom_Sore through,Symptom_Rhinorrhea,...,LAB_CR_1,LAB_NA_First,LAB_K_First,LAB_ALKP_First,LAB_ESR_First,LAB_CPK_First,LAB_PTT_First,LAB_PT_First,LAB_INR_First,Demographic_Age
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.143712,134.000000,0.579784,2.278754,1.792392,1.602060,1.478566,1.056905,0.021189,1.579784
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.531479,128.000000,0.792392,2.276462,1.986772,1.991226,1.585461,1.079181,-0.050610,1.949390
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.431364,125.000000,0.579784,2.294466,1.716003,2.730782,1.555094,1.283301,0.250420,1.903090
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.315309,137.027804,0.876406,2.348832,1.780855,2.522683,1.656591,1.297405,0.769455,1.819544
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.146128,134.000000,0.707570,2.303196,0.845098,2.281033,1.307496,1.110590,0.075547,1.623249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.264256,137.054587,0.887880,2.321383,1.764896,2.577037,1.649481,1.272190,0.739030,1.531479
6114,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.096910,140.000000,0.653213,2.340444,1.414973,1.602060,1.403121,1.037426,0.000000,1.662758
6115,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.258519,136.000000,0.568202,2.351238,2.485280,2.623930,1.576610,1.215748,0.510651,1.792392
6116,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.460158,137.370104,1.176594,2.449030,-0.212290,2.458249,1.756472,1.504318,1.000218,1.929419


In [621]:
def clean_data(X_train):
    # Replace missing values with the mean of each column in: 'L1_BloodGroup_First', 'Demographic_Gender' and 74 other columns
    X_train = X_train.fillna({ 'Demographic_Gender': X_train['Demographic_Gender'].mean(), 'Symptom_Caugh': X_train['Symptom_Caugh'].mean(), 'Symptom_Dyspnea': X_train['Symptom_Dyspnea'].mean(), 'Symptom_Fever': X_train['Symptom_Fever'].mean(), 'Symptom_Chiver': X_train['Symptom_Chiver'].mean(), 'Symptom_Mylagia': X_train['Symptom_Mylagia'].mean(), 'Symptom_Weakness': X_train['Symptom_Weakness'].mean(), 'Symptom_LOC': X_train['Symptom_LOC'].mean(), 'Symptom_Sore through': X_train['Symptom_Sore through'].mean(), 'Symptom_Rhinorrhea': X_train['Symptom_Rhinorrhea'].mean(), 'Symptom_Smelling disorder': X_train['Symptom_Smelling disorder'].mean(), 'Symptom_nauseaVomit': X_train['Symptom_nauseaVomit'].mean(), 'Symptom_Anorexia': X_train['Symptom_Anorexia'].mean(), 'Symptom_Diarhhea': X_train['Symptom_Diarhhea'].mean(), 'Symptom_ChestPain': X_train['Symptom_ChestPain'].mean(), 'Symptom_Seizure': X_train['Symptom_Seizure'].mean(), 'Symptom_SkinLesion': X_train['Symptom_SkinLesion'].mean(), 'Symptom_Jointpain': X_train['Symptom_Jointpain'].mean(), 'Symptom_Headache': X_train['Symptom_Headache'].mean(), 'Symptom_AbdominalPain': X_train['Symptom_AbdominalPain'].mean(), 'Symptom_Earpain': X_train['Symptom_Earpain'].mean(), 'Symptom_Hemorrhasia': X_train['Symptom_Hemorrhasia'].mean(), 'Symptom_Hemiparesia': X_train['Symptom_Hemiparesia'].mean(), 'MH_PregnanAcy': X_train['MH_PregnanAcy'].mean(), 'MH_CurremtSmoker': X_train['MH_CurremtSmoker'].mean(), 'MH_Alcoholuser': X_train['MH_Alcoholuser'].mean(), 'MH_Opiumuser': X_train['MH_Opiumuser'].mean(), 'MH_Hookahuser': X_train['MH_Hookahuser'].mean(), 'MH_HTN': X_train['MH_HTN'].mean(), 'MH_IHD': X_train['MH_IHD'].mean(), 'MH_CABG': X_train['MH_CABG'].mean(), 'MH_CHF': X_train['MH_CHF'].mean(), 'MH_Ashtma': X_train['MH_Ashtma'].mean(), 'MH_COPD': X_train['MH_COPD'].mean(), 'MH_DM': X_train['MH_DM'].mean(), 'MH_Pneumonia': X_train['MH_Pneumonia'].mean(), 'MH_CVA': X_train['MH_CVA'].mean(), 'MH_GIdisorder': X_train['MH_GIdisorder'].mean(), 'MH_CKD': X_train['MH_CKD'].mean(), 'MH_RA': X_train['MH_RA'].mean(), 'Cancer': X_train['Cancer'].mean(), 'MH_HLP': X_train['MH_HLP'].mean(), 'MH_Hep C': X_train['MH_Hep C'].mean(), 'MH_Thyroid dysfunction': X_train['MH_Thyroid dysfunction'].mean(), 'MH_Immunocompromised': X_train['MH_Immunocompromised'].mean(), 'MH_ChronicSeizure': X_train['MH_ChronicSeizure'].mean(), 'MH_TB': X_train['MH_TB'].mean(), 'MH_Anemia': X_train['MH_Anemia'].mean(), 'MH_Fattyliver': X_train['MH_Fattyliver'].mean(), 'MH_Psychologicaldisorder': X_train['MH_Psychologicaldisorder'].mean(), 'MH_Parkinson': X_train['MH_Parkinson'].mean(), 'MH_Alzhimer': X_train['MH_Alzhimer'].mean(), 'symtpm_to_referral': X_train['symtpm_to_referral'].mean(), 'VS_O2satwithoutsupp': X_train['VS_O2satwithoutsupp'].mean(), 'VS_PR': X_train['VS_PR'].mean(), 'VS_diastolic BP': X_train['VS_diastolic BP'].mean(), 'VS_Systolic BP': X_train['VS_Systolic BP'].mean(), 'VS_RR': X_train['VS_RR'].mean(), 'VS_T': X_train['VS_T'].mean(), 'LAB_WBC_1': X_train['LAB_WBC_1'].mean(), 'LAB_LYMPHH_1': X_train['LAB_LYMPHH_1'].mean(), 'LAB_NEUT_1': X_train['LAB_NEUT_1'].mean(), 'LAB_PLT_1': X_train['LAB_PLT_1'].mean(), 'LAB_HB_1': X_train['LAB_HB_1'].mean(), 'LAB_MCV_1': X_train['LAB_MCV_1'].mean(), 'LAB_CR_1': X_train['LAB_CR_1'].mean(), 'LAB_NA_First': X_train['LAB_NA_First'].mean(), 'LAB_K_First': X_train['LAB_K_First'].mean(), 'LAB_ALKP_First': X_train['LAB_ALKP_First'].mean(), 'LAB_ESR_First': X_train['LAB_ESR_First'].mean(), 'LAB_CPK_First': X_train['LAB_CPK_First'].mean(), 'LAB_PTT_First': X_train['LAB_PTT_First'].mean(), 'LAB_PT_First': X_train['LAB_PT_First'].mean(), 'LAB_INR_First': X_train['LAB_INR_First'].mean(), 'Demographic_Age': X_train['Demographic_Age'].mean()})
    return X_train

X_train = clean_data(X_train.copy())
X_test = clean_data(X_test.copy())
X_ex = clean_data(X_ex.copy())


In [622]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

transform = StandardScaler()

def standard(f):
    df_ex_normalize = transform.fit_transform(f)
    df_ex_normalize = pd.DataFrame(df_ex_normalize)
    df_ex_normalize.columns = f.columns
    return df_ex_normalize


In [623]:
num = list(X_train.columns[X_train.columns.get_loc('VS_O2satwithoutsupp'): X_train.columns.get_loc('Demographic_Age') + 1]
)

X_train[num]=standard(X_train[num])
X_test[num]=standard(X_test[num])
X_ex[num]=standard(X_ex[num])

In [624]:
from imblearn.over_sampling import SMOTE

under_sampler = SMOTE()
X_train, y_train = under_sampler.fit_resample(X_train, y_train)
X_test, y_test = under_sampler.fit_resample(X_test, y_test)
X_ex, y_ex = under_sampler.fit_resample(X_ex, y_ex )


In [625]:
from sklearn.linear_model import Lasso

def lasso_MO_ICU(X_train, y_train,X_test , X_ex):

    
    lasso = Lasso(alpha=0.001,random_state=42 )
    lasso.fit(X_train, y_train)


    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_ex = pd.DataFrame(X_ex)

    selected_feature_indices = sorted_indices[:40]
    selected_feature_indices = np.where(lasso.coef_ != 0)[0]


    X_train_selected_Mortality_ICU = X_train.iloc[:, selected_feature_indices]
    X_test_selected_Mortality_ICU = X_test.iloc[:, selected_feature_indices]
    X_ex = X_ex.iloc[:, selected_feature_indices]
    return X_train_selected_Mortality_ICU ,X_test_selected_Mortality_ICU,X_ex, selected_feature_indices

X_train,X_test,X_ex, selected_feature_indices = lasso_MO_ICU(X_train, y_train,X_test,X_ex)


In [626]:
X_train

,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,Symptom_Chiver,Symptom_Mylagia,Symptom_Weakness,Symptom_LOC,Symptom_Sore through,Symptom_Smelling disorder,...,LAB_PLT_1,LAB_HB_1,LAB_MCV_1,LAB_CR_1,LAB_NA_First,LAB_ALKP_First,LAB_CPK_First,LAB_PT_First,LAB_INR_First,Demographic_Age
0,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.512831,-0.075833,0.170223,-0.246362,-0.610105,-0.153576,-1.684965,-0.743650,-0.748182,-0.891649
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.0,...,2.582486,2.219446,-0.350273,1.321215,-1.816609,-0.167348,-0.751171,-0.634299,-0.904458,1.181027
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,1.000000,0.0,0.0,...,0.717423,2.786070,-0.927876,0.916491,-2.419860,-0.059154,1.023377,0.367686,-0.249246,0.921386
3,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.159346,0.102614,-0.027264,0.447329,-0.001262,0.267546,0.524047,0.436916,0.880469,0.452877
4,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,-0.555559,1.226233,-2.566622,-0.236596,-0.610105,-0.006695,-0.055785,-0.480121,-0.629869,-0.647903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9755,0.000000,0.087775,0.087775,0.000000,0.000000,0.0,0.0,0.912225,0.0,0.0,...,0.104144,0.307122,-0.150075,0.541447,0.004773,0.381238,0.509196,0.515037,0.863646,0.943572
9756,0.227531,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,...,1.713974,0.725859,0.480637,0.055489,0.441068,0.023163,-0.952070,-0.083404,0.025356,1.002425
9757,0.000000,0.876366,0.123634,0.876366,0.876366,0.0,0.0,0.123634,0.0,0.0,...,0.343174,-0.304718,-0.690459,-0.787756,0.695842,-0.245008,-1.168538,-0.281567,-0.541071,0.861585
9758,1.000000,1.000000,1.000000,0.153491,0.000000,0.0,1.0,0.000000,0.0,0.0,...,0.068826,0.230954,-0.163645,0.483468,-0.001116,0.341750,0.514248,0.397659,0.828496,0.840158


In [627]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

In [628]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def calculate_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    cm = confusion_matrix(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted)
    recall = recall_score(y_test, y_predicted)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) != 0 else 0
    f1 = f1_score(y_test, y_predicted)
    
    return accuracy, precision, recall, specificity, f1

In [629]:
from sklearn.metrics import roc_auc_score

def calculate_and_update_model_metrics(y_test, y_predicted,y_ex, y_ex_predicted):
    accuracy, precision, recall, specificity, f1 = calculate_metrics(y_test, y_predicted)
    model = {}
    model["accuracy"] = accuracy
    model["precision"] = precision
    model["recall"] = recall
    model["specificity"] = specificity
    model["f1"] = f1
    model["AUC"] = roc_auc_score(y_test, y_predicted)
    
    accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex = calculate_metrics(y_ex, y_ex_predicted)
    model["accuracy_ex"] = accuracy_ex
    model["precision_ex"] = precision_ex
    model["recall_ex"] = recall_ex
    model["specificity_ex"] = specificity_ex
    model["f1_ex"] = f1_ex
    model["AUC_ex"] = roc_auc_score(y_test, y_predicted)
    return model


In [630]:
def logistic_regression_classifier(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = LogisticRegression(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)

    

    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [631]:
def train_and_evaluate_svm(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = svm.SVC(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)



    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [632]:
def train_and_evaluate_tree(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = DecisionTreeClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [633]:
def train_and_evaluate_knn(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {}
    lr = KNeighborsClassifier()
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [634]:
def train_and_evaluate_forest(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = RandomForestClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [635]:
from xgboost import XGBClassifier
def train_and_evaluate_boost(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = XGBClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [636]:
list_Outcome_InhospitalMortality=[]




In [637]:
y_train_ = np.array(y_train)
y_test_ = np.array(y_test)

y_ex_ = np.array(y_ex)




In [638]:

list_Outcome_InhospitalMortality.extend([logistic_regression_classifier(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                            train_and_evaluate_svm(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_tree(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_knn(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_forest(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_boost(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_)])


result_dic_list_Outcome_InhospitalMortality = dict(zip(['logistic_regression', 'svm', 'tree', 'knn', 'forest', 'boost'], list_Outcome_InhospitalMortality))





merged_dict = {}

# List of dictionary names and their corresponding dictionaries
dict_list = [('Outcome_InhospitalMortality', result_dic_list_Outcome_InhospitalMortality)]

# Merge the dictionaries
for name, result_dict in dict_list:
    merged_dict[name] = result_dict

# The merged_dict now contains all the dictionaries merged together


dff = pd.DataFrame(merged_dict)


dff = dff.transpose()


dff.reset_index(inplace=True)
dff.rename(columns={'index': 'Method'}, inplace=True)





data = []

for outcome, models in merged_dict.items():
    for model, metrics in models.items():
        accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex = metrics[0]['accuracy'], metrics[0]['precision'], metrics[0]['recall'], metrics[0]['specificity'], metrics[0]['f1'], metrics[0]['AUC'],metrics[0]['accuracy_ex'], metrics[0]['precision_ex'], metrics[0]['recall_ex'], metrics[0]['specificity_ex'], metrics[0]['f1_ex'], metrics[0]['AUC_ex']
        y_true, y_predicted,y_true_ex,y_predicted_ex = metrics[1]['y_true'], metrics[1]['y_predicted'],metrics[1]['y_true_ex'], metrics[1]['y_predicted_ex']
        data.append([outcome, model,accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex, y_true.tolist(), y_predicted.tolist(),y_true_ex.tolist(),y_predicted_ex.tolist()])

columns = ['Outcome', 'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1','AUC', 'Accuracy_ex', 'Precision_ex', 'Recall_ex', 'Specificity_ex', 'F1_ex','AUC_ex', 'y_true', 'y_predicted', 'y_true_ex', 'y_predicted_ex']

dff = pd.DataFrame(data, columns=columns)



In [639]:
dff

,Outcome,Model,Accuracy,Precision,Recall,Specificity,F1,AUC,Accuracy_ex,Precision_ex,Recall_ex,Specificity_ex,F1_ex,AUC_ex,y_true,y_predicted,y_true_ex,y_predicted_ex
0,Outcome_InhospitalMortality,logistic_regression,0.733603,0.726630,0.748988,0.718219,0.737640,0.733603,0.736210,0.729870,0.750000,0.722420,0.739798,0.733603,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ..."
1,Outcome_InhospitalMortality,svm,0.747368,0.718370,0.813765,0.680972,0.763098,0.747368,0.727758,0.713333,0.761566,0.693950,0.736661,0.747368,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,Outcome_InhospitalMortality,tree,0.762348,0.743609,0.800810,0.723887,0.771150,0.762348,0.745107,0.729011,0.780249,0.709964,0.753760,0.762348,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ..."
3,Outcome_InhospitalMortality,knn,0.683401,0.690496,0.664777,0.702024,0.677393,0.683401,0.674822,0.686610,0.643238,0.706406,0.664217,0.683401,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
4,Outcome_InhospitalMortality,forest,0.859919,0.847537,0.877733,0.842105,0.862371,0.859919,0.813612,0.784964,0.863879,0.763345,0.822533,0.859919,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
5,Outcome_InhospitalMortality,boost,0.862348,0.888792,0.828340,0.896356,0.857502,0.862348,0.812722,0.842259,0.769573,0.855872,0.804277,0.862348,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, ..."
